# Part 2 / LIME Analysis

LIME (Local Interpretable Model-agnostic Explanations) Analysis
Question 6 - Part 2: Local interpretability analysis

This notebook implements LIME explanations for individual predictions
of the XGBoost credit scoring model.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import string
from lime.lime_tabular import LimeTabularExplainer
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def get_features(df: pd.DataFrame()) -> pd.DataFrame():
    """Get additional features - matching model training preprocessing."""
    df_with_features = (
        df
        .assign(
            # Only revol_bal_log (not other logs)
            revol_bal_log = np.log1p(df["revol_bal"]),

            # total balance
            cur_balance = df["avg_cur_bal"] * df["open_acc"],

            # flags
            delinq_2yrs_flag = df["delinq_2yrs"] >= 1,
            tax_liens_flag = df["tax_liens"] >= 1,

            # shares
            s_actv_bc_tl = df["num_actv_bc_tl"] / (df["open_acc"] + 1e-6),
            s_bc_tl = df["num_bc_tl"] / (df["open_acc"] + 1e-6),
            s_il_tl = df["num_il_tl"] / (df["open_acc"] + 1e-6),
            s_rev_accts = df["num_rev_accts"] / (df["open_acc"] + 1e-6),

            # interactions
            revol_bal_income_ratio = df["revol_bal"] / (df["annual_inc"] + 1e-6),
        )
    )
    return df_with_features


In [ ]:
def categorical_encoding(df: pd.DataFrame) -> pd.DataFrame:
    """Encodings of categorical variables - matching model training preprocessing."""
    df_encoded = df.copy()

    # sub_grade to numeric
    sg = df_encoded["sub_grade"].astype(str).str.upper().str.strip()
    letter = sg.str[0]
    number = pd.to_numeric(sg.str[1:].str.extract(r"(\d+)", expand=False), errors="coerce")
    letter_map = {ch: i+1 for i, ch in enumerate("ABCDEFG")}
    base = letter.map(letter_map)
    sub_grade_num = (base - 1) * 5 + number
    df_encoded["sub_grade_num"] = sub_grade_num.astype("float32")

    # emp_length to numeric
    emp_length_map = {
        '< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
        '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9,
        '10+ years': 10
    }
    df_encoded["emp_length_num"] = df_encoded["emp_length"].map(emp_length_map).astype("float32")

    # zip_risk encoding (instead of zip_code2)
    # Create zip_risk categories based on zip_code
    df_encoded["zip_risk"] = pd.cut(df_encoded["zip_code"], 
                                   bins=[0, 20000, 40000, 60000, 100000], 
                                   labels=["Low", "Low-Med", "Medium", "Med-High"])
    df_encoded = pd.get_dummies(df_encoded, columns=["zip_risk"], prefix="zip_risk")

    # one-hot for home_ownership and purpose
    onehot_cols = ["home_ownership", "purpose"]
    df_encoded = pd.get_dummies(df_encoded, columns=onehot_cols, prefix=onehot_cols, drop_first=True)

    # emp_title with grouped prefix (matching model)
    df_encoded = pd.get_dummies(df_encoded, columns=["emp_title"], prefix="emp_title_grouped", drop_first=True)

    # drop originals and unused columns
    df_encoded = df_encoded.drop(columns=["grade", "sub_grade", "emp_length", "issue_d", "revol_bal", "zip_code", "Pct_afro_american"])

    return df_encoded


In [4]:
# Load the model
xgb_model = joblib.load('optimized_xgb_model.pkl')

# Load and prepare data
df = pd.read_csv("dataproject2025.csv", index_col=0)
df.drop(columns=['Predictions', 'Predicted probabilities'], inplace=True)
df_dropped = df.dropna(axis=0)

# Feature engineering
df_engineered = get_features(df_dropped)
df_encoded = categorical_encoding(df_engineered)

# Prepare features and target
target_col = 'target'
X = df_encoded.drop(columns=[target_col])
y = df_encoded[target_col]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Data loaded successfully!")
print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")


Data loaded successfully!
Training set size: (868988, 112)
Test set size: (217248, 112)


In [ ]:
# LIME explanations on representative test instances
# Use training distribution to fit the explainer
X_train_np = X_train.values if hasattr(X_train, 'values') else np.asarray(X_train)
X_test_np = X_test.values if hasattr(X_test, 'values') else np.asarray(X_test)

class_names = ['no_default', 'default']
feature_names_list = list(X_train.columns)

explainer = LimeTabularExplainer(
    training_data=X_train_np,
    mode='classification',
    feature_names=feature_names_list,
    class_names=class_names,
    discretize_continuous=True,
    sample_around_instance=True,
    verbose=False,
    random_state=42,
)

print("LIME explainer initialized successfully!")
print(f"Number of features: {len(feature_names_list)}")
print(f"Training samples: {X_train_np.shape[0]}")
print(f"Test samples: {X_test_np.shape[0]}")


In [ ]:
# Pick representative instances to explain
# Select instances with different predicted probabilities
y_pred_proba = xgb_model.predict_proba(X_test_np)[:, 1]

# Find instances with different risk levels
low_risk_idx = np.where((y_pred_proba >= 0.1) & (y_pred_proba <= 0.3))[0][:1]  # Low risk
medium_risk_idx = np.where((y_pred_proba >= 0.4) & (y_pred_proba <= 0.6))[0][:1]  # Medium risk  
high_risk_idx = np.where((y_pred_proba >= 0.7) & (y_pred_proba <= 0.9))[0][:1]  # High risk

indices_to_explain = np.concatenate([low_risk_idx, medium_risk_idx, high_risk_idx])

print("Selected instances for LIME analysis:")
for i, idx in enumerate(indices_to_explain):
    actual_label = y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]
    pred_proba = y_pred_proba[idx]
    print(f"Instance {i+1} (Index {idx}): Predicted probability = {pred_proba:.3f}, Actual label = {actual_label}")


In [ ]:
# Generate LIME explanations
def predict_fn(data):
    """Prediction function for LIME"""
    return xgb_model.predict_proba(data)

lime_results = []
for idx in indices_to_explain:
    instance = X_test_np[idx]
    exp = explainer.explain_instance(
        data_row=instance,
        predict_fn=predict_fn,
        num_features=10,  # Top 10 features
        top_labels=2,     # Both classes
    )
    lime_results.append((idx, exp))

print("LIME explanations generated successfully!")
print(f"Analyzed {len(lime_results)} instances")


In [ ]:
# Display LIME explanations
for i, (idx, exp) in enumerate(lime_results):
    actual_label = y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]
    pred_proba = y_pred_proba[idx]
    
    print(f"\n{'='*60}")
    print(f"LIME EXPLANATION - Instance {i+1} (Index {idx})")
    print(f"{'='*60}")
    print(f"Predicted probability of default: {pred_proba:.3f}")
    print(f"Actual label: {'Default' if actual_label == 1 else 'No Default'}")
    print(f"Prediction: {'High Risk' if pred_proba > 0.5 else 'Low Risk'}")
    
    # Get explanation for the predicted class
    predicted_class = 1 if pred_proba > 0.5 else 0
    explanation = exp.as_list(label=predicted_class)
    
    print(f"\nTop 10 features contributing to prediction:")
    print("-" * 50)
    for feature, weight in explanation:
        print(f"{feature:<30} {weight:>8.4f}")
    
    print(f"\nFeature values for this instance:")
    print("-" * 50)
    instance_data = X_test.iloc[idx] if hasattr(X_test, 'iloc') else pd.Series(X_test_np[idx], index=feature_names_list)
    for feature, _ in explanation[:5]:  # Show top 5 features
        if feature in instance_data.index:
            print(f"{feature:<30} {instance_data[feature]:>8.4f}")


In [ ]:
# Visualize LIME explanations
fig, axes = plt.subplots(1, len(lime_results), figsize=(15, 5))
if len(lime_results) == 1:
    axes = [axes]

for i, (idx, exp) in enumerate(lime_results):
    actual_label = y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]
    pred_proba = y_pred_proba[idx]
    predicted_class = 1 if pred_proba > 0.5 else 0
    
    # Create LIME plot manually
    explanation = exp.as_list(label=predicted_class)
    
    # Extract features and weights
    features = [x[0] for x in explanation]
    weights = [x[1] for x in explanation]
    
    # Create horizontal bar plot
    y_pos = np.arange(len(features))
    colors = ['red' if w > 0 else 'green' for w in weights]
    
    axes[i].barh(y_pos, weights, color=colors, alpha=0.7)
    axes[i].set_yticks(y_pos)
    axes[i].set_yticklabels(features)
    axes[i].set_xlabel('Feature Weight')
    axes[i].axvline(x=0, color='black', linestyle='-', alpha=0.3)
    axes[i].set_title(f'Instance {i+1}: P(default)={pred_proba:.3f}\nActual: {"Default" if actual_label==1 else "No Default"}')
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Part 2 / LIME Analysis: Local Explanations for Individual Predictions', y=1.02)
plt.tight_layout()
plt.show()


## LIME Analysis Summary

### What LIME Shows Us:

**1. Local Interpretability**
- LIME explains **why** the model made a specific prediction for individual instances
- Each explanation shows which features contributed most to that particular decision
- Features are ranked by their importance for that specific prediction

**2. Feature Contributions**
- **Positive weights**: Features that increase the probability of default
- **Negative weights**: Features that decrease the probability of default
- **Magnitude**: How much each feature influenced the prediction

**3. Business Value**
- **Transparency**: Explain individual loan decisions to customers
- **Debugging**: Understand why certain predictions seem wrong
- **Validation**: Verify that the model uses sensible features for decisions
- **Compliance**: Meet regulatory requirements for explainable AI

### Key Insights:
- Different instances can have very different explanations even with similar predictions
- The same feature can have opposite effects depending on the individual's profile
- LIME helps identify which specific factors drove each lending decision
